In [1]:
import numpy as np
import pandas as pd

import refinitiv.dataplatform.eikon as ek
import option_ric_tools as ort

ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

Let's start by defining what data we will be fetching. We will need to define the following:
* The ISIN for the underlying security as a string
* The maturity date for any options as a string
* The strike prices for which to fetch options prices as a list of integers
* The kinds of options (call/put) to fetch prices for as a list of strings (allowed values are 'C' and 'P')

An example for IBM is provided below. If you are unsure what strikes to fetch you might want to check the price of the underlying 90 days prior to your chosen maturity. Any configurations for which no prices are available will be skipped.

In [2]:
isin = 'US4592001014'
maturity = '2022-03-18'
strikes = [115, 120, 125, 130, 135]
kinds = ['C', 'P']

The code below will then try to fetch pricing for what we defined. Using the Eikon API requires quite a lot of preprocessing, which is taken care of in the file 'option_ric_tools.py' The price series for each option will be added as a column.

In [3]:
price_dfs = []
for kind in kinds:
    assert kind in ['C', 'P']
    for strike in strikes:
        kind_str = 'call' if kind == 'C' else 'P'
        print(f'\nFetching prices for {kind_str} option with strike price {strike}')
        rics, prices_list = ort.get_optionRic(isin, maturity, strike, kind)
        try:
            price_df = prices_list[0]
            price_df = price_df.drop(['BID', 'ASK'], axis=1)
            price_df.columns = [f'{kind}{strike}']
            price_dfs.append(price_df)
        except IndexError:
            print('Skipping empty combination')

price_df = pd.concat(price_dfs, axis=1)


Fetching prices for call option with strike price 115
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 120
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 125
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 130
Option RIC for OPQ exchange is successfully constructed

Fetching prices for call option with strike price 135
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 115
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 120
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 125
Option RIC for OPQ exchange is successfully constructed

Fetching prices for P option with strike price 130
Option RIC for OPQ exchange is successfully constructed

Fetching pri

In [4]:
price_df.head()

C115   C120  C125  C130  C135  P115  P120  P125  P130  P135
2021-12-20  <NA>   9.81   6.9  4.44  2.81  2.84  3.95   6.6  9.45  <NA>
2021-12-21  <NA>   11.6  7.91     5  3.15  2.05   3.1   4.9  7.45  <NA>
2021-12-22  <NA>   11.6  8.06   5.2  3.15  1.77  2.85  4.36   6.9  <NA>
2021-12-23  <NA>   12.7   8.4   5.7  3.45  1.52  2.44  3.75  5.95  8.87
2021-12-27  <NA>  13.05   9.1  5.75   3.5  1.36  2.12  3.45   5.4   8.5

Let's now fetch the price series for the underlying for the same time period. Eikon uses a special identifier called RIC. The code below find the corresponding RIC for the ISIN we defined. It also find the earliest and latest dates in our options price series.

In [5]:
ric = ek.get_symbology(symbol=isin, from_symbol_type='ISIN', to_symbol_type='RIC').iloc[0,0]
sdate = str(price_df.index.min().date())
edate = str(price_df.index.max().date())

We can now fetch the price series for the underlying and add it as a column to the dataframe.

In [6]:
stock = ek.get_timeseries(
    rics=[ric],
    fields=['CLOSE'],
    start_date=sdate,
    end_date=edate,
    interval='daily'
)

stock = stock.reset_index()
stock.columns.name = None
stock.index = stock['Date']
stock.index.name = None
stock = stock.drop('Date', axis=1)
stock.columns = ['Underlying']

price_df = stock.join(price_df)

In [7]:
price_df.head()

Underlying  C115   C120  C125  C130  C135  P115  P120  P125  P130  \
2021-12-20      127.06  <NA>   9.81   6.9  4.44  2.81  2.84  3.95   6.6  9.45   
2021-12-21      128.97  <NA>   11.6  7.91     5  3.15  2.05   3.1   4.9  7.45   
2021-12-22      129.75  <NA>   11.6  8.06   5.2  3.15  1.77  2.85  4.36   6.9   
2021-12-23      130.63  <NA>   12.7   8.4   5.7  3.45  1.52  2.44  3.75  5.95   
2021-12-27      131.62  <NA>  13.05   9.1  5.75   3.5  1.36  2.12  3.45   5.4   

            P135  
2021-12-20  <NA>  
2021-12-21  <NA>  
2021-12-22  <NA>  
2021-12-23  8.87  
2021-12-27   8.5

As a final step we save the constructed dataframe for further analysis. The default code saves the dataframe in feather format, but if you can issues you can uncomment the lines further down to export to csv.

In [8]:
price_df.index.name = 'Date'
price_df = price_df.reset_index()
price_df.to_feather('fe2.feather')
#price_df.to_csv('fe2.csv')